In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install mne
import matplotlib.pyplot as plt
import mne
from mne.io import concatenate_raws, read_raw_edf
import numpy as np
import pandas as pd


     |████████████████████████████████| 7.4 MB 6.7 MB/s 


In [ ]:
import os
list_a = [] #存放所有.edf文件的路径
dir_str = '/content/drive/MyDrive/EEG project/MIdataset/files'
task_num = ['04','08','12']
for i in range(1,41):
  if i < 10:
    i = '0'+str(i)
  else:
    i = str(i)
  for j in task_num:
    a = os.path.join(dir_str, "S0"+i, "S0"+i+'R'+j+'.edf')
    list_a.append(a)

In [ ]:
len(list_a)

120

In [ ]:

def get_data(p_list):
  X,y = [],[]
  for i in p_list:
    raw=read_raw_edf(i,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    raw.pick_channels(['Cz..','C3..','C4..'])
    epochs = mne.Epochs(raw, events=events_from_annot, event_id=event_dict, tmin=-2, tmax=2,
                      baseline=None, preload=True)
    labels = list(event_dict.keys())
    for l in labels:
        for epoch in epochs[l]:
          e = np.delete(epoch,-1,axis=1)  #把641变成640
          j = e.reshape((3,32,20))
          X.append(j)
          y.append(event_dict[l]-1)#..
  y = np.array(y)
  X = np.array(X)
  return (X,y)

train_X,train_y = get_data(list_a[:90])
test_X, test_y = get_data(list_a[90:])
# for i in list_a[:30]:
#   raw=read_raw_edf(i,preload=False)
#   events_from_annot, event_dict = mne.events_from_annotations(raw)
#   raw.pick_channels(['Cz..','C3..','C4..'])
#   epochs = mne.Epochs(raw, events=events_from_annot, event_id=event_dict, tmin=-2, tmax=2,
#                     baseline=None, preload=True)
#   labels = list(event_dict.keys())
#   for l in labels:
#       for epoch in epochs[l]:
#         e = np.delete(epoch,-1,axis=1)  #把641变成640
#         j = e.reshape((3,32,20))
#         X.append(j)
#         y.append(event_dict[l]-1)#..

Extracting EDF parameters from /content/drive/MyDrive/EEG project/MIdataset/files/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., Cz.., C2.., C4.., C6.., Cp5., Cp3., Cp1., Cpz., Cp2., Cp4., Cp6., Fp1., Fpz., Fp2., Af7., Af3., Afz., Af4., Af8., F7.., F5.., F3.., F1.., Fz.., F2.., F4.., F6.., F8.., Ft7., Ft8., T7.., T8.., T9.., T10., Tp7., Tp8., P7.., P5.., P3.., P1.., Pz.., P2.., P4.., P6.., P8.., Po7., Po3., Poz., Po4., Po8., O1.., Oz.., O2.., Iz..
Creating raw.info structure...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Loading data for 30 events and 641 original time points ...
1 bad epochs dropped
Extracting EDF parameters from /content/drive/MyDrive/EEG project/MIdataset/files/S001/S0

In [9]:
%cd '/content/drive/MyDrive/EEG project'
data = np.load(file="rgb_eeg.npy")
labels = np.load(file='labels.npy')
train_X= data[:6090]
train_y= labels[:6090]
test_X= data[6090:]
test_y= labels[6090:]

/content/drive/MyDrive/EEG project


In [10]:
print(test_X.shape)
print(test_y.shape)

(870, 32, 20, 3)
(870,)


In [11]:
import torch
import pandas as pd
from torch import nn,optim
import torchvision
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable


In [14]:
class MIDataset(torch.utils.data.Dataset):
	#dataset 
    def __init__(self, input_data, labels, transform=None):
      self.input_data = input_data
      self.labels = labels
      self.transform = transform
 
    def __len__(self):
      return len(self.labels)
 
    def __getitem__(self, index):
      data = self.input_data[index]
      label = self.labels[index]
      if self.transform is not None:
        data = self.transform(data)
      return data,label

In [19]:
mean = [0.485, 0.456, 0.406]
std = [0.228, 0.224, 0.225]
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

transform_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [20]:
train_data = MIDataset(train_X,train_y,transform_train)
test_data = MIDataset(test_X,test_y,transform_test)

In [21]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=64,shuffle=False)

In [ ]:
modellr = 1e-4
BATCH_SIZE = 64
EPOCHS = 20
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 实例化模型并且移动到GPU
model = models.resnet18(pretrained=False)
numFit = model.fc.in_features
numClass = 3
model.fc = nn.Sequential(nn.Linear(numFit, numClass), nn.Softmax(dim=1))
criterion = nn.CrossEntropyLoss()
model.to(DEVICE)
# 选择简单暴力的Adam优化器，学习率调低
optimizer = optim.Adam(model.parameters(), lr=modellr)

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew


In [ ]:
# 定义训练过程

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_loss = 0
    total_num = len(train_loader.dataset)
    print(total_num, len(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data.float())
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
        if (batch_idx + 1) % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))
    ave_loss = sum_loss / len(train_loader)
    print('epoch:{},loss:{}'.format(epoch, ave_loss))

def val(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total_num = len(test_loader.dataset)
    print(total_num, len(test_loader))
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data.float())
            loss = criterion(output, target)
            _, pred = torch.max(output.data, 1)
            correct += torch.sum(pred == target)
            print_loss = loss.data.item()
            test_loss += print_loss
        correct = correct.data.item()
        acc = correct / total_num
        avgloss = test_loss / len(test_loader)
        print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc))




for epoch in range(1, EPOCHS + 1):
    adjust_learning_rate(optimizer, epoch)
    train(model, DEVICE, train_loader, optimizer, epoch)
    val(model, DEVICE, test_loader)

lr: 0.0001
2610 41
epoch:1,loss:1.0217387763465322
870 14

Val set: Average loss: 1.0530, Accuracy: 420/870 (48%)

lr: 0.0001
2610 41
epoch:2,loss:0.9422060963584156
870 14

Val set: Average loss: 1.0495, Accuracy: 404/870 (46%)

lr: 0.0001
2610 41
epoch:3,loss:0.8662370894013381
870 14

Val set: Average loss: 0.9906, Accuracy: 459/870 (53%)

lr: 0.0001
2610 41
epoch:4,loss:0.7979512694405346
870 14

Val set: Average loss: 1.0205, Accuracy: 434/870 (50%)

lr: 0.0001
2610 41
epoch:5,loss:0.7588228089053456
870 14

Val set: Average loss: 0.9980, Accuracy: 467/870 (54%)

lr: 0.0001
2610 41
epoch:6,loss:0.7259941944261876
870 14

Val set: Average loss: 1.0487, Accuracy: 409/870 (47%)

lr: 0.0001
2610 41
epoch:7,loss:0.70545422013213
870 14

Val set: Average loss: 1.0183, Accuracy: 452/870 (52%)

lr: 0.0001
2610 41
epoch:8,loss:0.693946922697672
870 14

Val set: Average loss: 1.0257, Accuracy: 447/870 (51%)

lr: 0.0001
2610 41
epoch:9,loss:0.6707286442198405
870 14

Val set: Average loss: 1